# Nlp final project

1. Yaniv Gabay - 205745615 - yanivga@edu.hac.ac.il
2. Shahar Asher - 209305408 - shaharas@edu.hac.ac.il
3. Hadar Liel Harush - 211721568 - hadarhar@edu.hac.ac.il
## Eminem song generator

This project will compare different models, all trained on the legendary hip-hop artist Eminem lyrics.
We want to expirment with different models, our own model, different apis, to see and compare the results.


to be added?

In [1]:
# Essential imports for handling arrays, dataframes, and neural network functionalities
import os
import torch
import pandas as pd

# Importing models and tokenizers from the Hugging Face Transformers library
from transformers import GPT2Model, GPT2Tokenizer

# Natural Language Processing (NLP) tools
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

# Downloading necessary NLTK data, consider running these lines only once
nltk.download('wordnet', quiet=True)  # WordNet Lemmatizer
nltk.download('punkt', quiet=True)    # Punkt Tokenizer models

# Our utilities file
import utilities as utils
# Optional: Uncomment the following line for CUDA debugging
# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.3/388.3 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 38.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 70.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.6/202.6 kB 27.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 53.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 38.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 k

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [2]:
##### Loading the data
### we be using <startVerse> <endVerse> <startChorus><endChorus>
## to give our model some different between the verses and the choruses.


##### you can change this to export the data
##### through the process 
debug = False
file_loc = './Eminem_Lyrics.csv'
df = utils.import_data_from_location(file_loc)

utils.export_data_to_csv(df,'afterImport-1',debug)



Error with encoding utf-8: 'utf-8' codec can't decode byte 0x92 in position 6: invalid start byte
Success with encoding: latin1
                        Album_Name       Song_Name  \
0  Music To Be Murdered By: Side B  Alfred (Intro)   
1  Music To Be Murdered By: Side B     Black Magic   
2  Music To Be Murdered By: Side B  Alfreds Theme   
3  Music To Be Murdered By: Side B       Tone Deaf   
4  Music To Be Murdered By: Side B  Book of Rhymes   

                                              Lyrics  \
0  [Intro: Alfred Hitchcock]\nThus far, this albu...   
1  [Chorus: Skylar Grey & Eminem]\nBlack magic, n...   
2  [Verse 1]\nBefore I check the mic (Check, chec...   
3  [Intro]\nYeah, I'm sorry (Huh?)\nWhat did you ...   
4  [Intro]\nI don't smile, I don't frown, get too...   

                                           Album_URL   Views  \
0  https://genius.com/albums/Eminem/Music-to-be-m...   24.3K   
1  https://genius.com/albums/Eminem/Music-to-be-m...  180.6K   
2  https://genius.

In [2]:
#actual messing with the dataset
import re


def insert_tokens(lyrics):

    """
    Inserts special tokens at the start and end of each verse and chorus in the lyrics.
    
    Parameters:
        lyrics (str): The lyrics of a song.
        
    Returns:
        str: The processed lyrics with inserted tokens.
    """
    # Pattern to identify all tags (verses, choruses, etc.)
    pattern = re.compile(r'\[(Verse|Chorus|Intro|Outro)')

    # Placeholder for processed lyrics
    processed_lyrics = "<startSong>\n"
    last_tag = None

    for line in lyrics.split('\n'):
        tag_match = pattern.match(line)
        if tag_match:
            # Close the previous tag if exists
            if last_tag:
                processed_lyrics += f"<end{last_tag}>\n"
            # Update last_tag and open a new tag
            last_tag = tag_match.group(1)  # Capture the tag name (Verse or Chorus)
            processed_lyrics += f"<start{last_tag}>\n"
        else:
            processed_lyrics += line + "\n"

    # Close the last opened tag
    if last_tag:
        processed_lyrics += f"<end{last_tag}>\n"
    processed_lyrics += "<endSong>"

    return processed_lyrics




## some songs can have some another tags, like post-chorus,pre etc.
## we arent intresnted in those, so we will just delete them in the next step
new_df = df['Lyrics'].apply(insert_tokens)
new_df = new_df.apply(utils.remove_tags)
new_df = new_df.apply(utils.handle_special_characters)
#lowercase everything
new_df = new_df.str.lower()

utils.export_data_to_csv(new_df,'afterLowerAndRemove',debug)


IndentationError: unindent does not match any outer indentation level (<tokenize>, line 17)

In [4]:

clean = new_df.apply(utils.remove_non_ascii_characters)
clean = new_df.apply(utils.expand_contractions, args=(utils.contractions_dict,))

utils.export_data_to_csv(clean,'afterRemoveNonAscii',debug)

#### we still have (lyrics) in the dataset, need to decide
### we still have ! or ? in the dataset, should we remove?

Removed characters: 
Removed characters: á
Removed characters: 
Removed characters: 
Removed characters: ó
Removed characters: 
Removed characters: 
Removed characters: éé
Removed characters: 
Removed characters: 
Removed characters: 
Removed characters: ö
Removed characters: 
Removed characters: 
Removed characters: 
Removed characters: 
Removed characters: 
Removed characters: 
Removed characters: 
Removed characters: éâé
Removed characters: ó
Removed characters: 
Removed characters: áó
Removed characters: 
Removed characters: 
Removed characters: 
Removed characters: 
Removed characters: 
Removed characters: 
Removed characters: óéáí
Removed characters: 
Removed characters: ú
Removed characters: 
Removed characters: â
Removed characters: 
Removed characters: 
Removed characters: 
Removed characters: 
Removed characters: 
Removed characters:

## we will try several tokenizer options

In [5]:
temp_switch = {'<startverse>':'startverse',
                '<endverse>':'endverse',
                '<startchorus>':'startchorus',
                '<endchorus>':'endchorus',
                '<startoutro>':'startoutro',
                '<endoutro>':'endoutro',
                '<startintro>':'startintro',
                '<endintro>':'endintro',
                '<startsong>':'startsong',
                '<endsong>':'endsong'}
def replace_tags(text, temp_switch, reverse=False):
    if reverse:
        temp_switch = {v: k for k, v in temp_switch.items()}

    for tag, replacement in temp_switch.items():
        text = text.replace(tag, replacement)
    return text

temp_clean = clean.apply(lambda x: replace_tags(x, temp_switch, reverse=False))
utils.export_data_to_csv(temp_clean,'temp_clean',debug)
!pip install spacy
import spacy
!python -m spacy download en_core_web_sm
nlp = spacy.load("en_core_web_sm")
spacy_tokens_df = temp_clean.apply(lambda x: [token.text for token in nlp(x)])


import nltk
from nltk.tokenize import word_tokenize
#nltk.download('punkt')
#nltk_tokens_df = temp_clean.apply(word_tokenize)



#total_token_models = [spacy_tokens_df, nltk_tokens_df, BERT_tokens_df, gpt2_tokens_df]
#for model in total_token_models:
  #  print("")
   # print(model.head())





##### now , we need to perpare the data for the embeddings
### so padding, indexing unique tokens, etc.

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 29.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [45]:
def build_vocab(series):
    """
    Builds a vocabulary dictionary mapping each unique token to a unique index.
    
    Special tokens are predefined with specific indices. This function iterates through
    each song in the provided series, updating the set of unique tokens. It then assigns
    an index to each token, starting with indices following the special tokens.
    
    Parameters:
        series (pd.Series): A pandas series containing the tokenized lyrics of songs.
        
    Returns:
        dict: A dictionary mapping tokens to their respective indices in the vocabulary.
    """
    # Initialize a set to store unique tokens found in the series
    unique_tokens = set()

    # Update the set with tokens from each song
    for song in series:
        unique_tokens.update(song.split())

    # Predefined special tokens with their indices
    token_to_idx = {
        '<PAD>': 0,
        '<startSong>': 1,
        '<endSong>': 2,
        '<startVerse>': 3,
        '<endVerse>': 4,
        '<startChorus>': 5,
        '<endChorus>': 6,
        '<startIntro>': 7,
        '<endIntro>': 8
    }

    # Assign indices to unique tokens, starting after the last special token
    for token in unique_tokens:
        if token not in token_to_idx:
            token_to_idx[token] = len(token_to_idx)

    return token_to_idx



def tokens_to_indices(tokenized_song, token_to_idx):
    """
    Converts a list of tokens from a song to their corresponding indices based on
    a given vocabulary mapping.
    
    If a token is not found in the mapping, it is replaced by the index of the '<PAD>'
    token, serving as a placeholder for unknown tokens.
    
    Parameters:
        tokenized_song (list of str): A list containing the tokens of a song.
        token_to_idx (dict): A dictionary mapping tokens to their respective indices.
        
    Returns:
        list of int: A list of indices corresponding to the tokens of the input song.
    """
    # Convert each token to its index, defaulting to the '<PAD>' token's index if not found
    return [token_to_idx.get(token, token_to_idx['<PAD>']) for token in tokenized_song]



In [46]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset

class TokenSongsDataset(Dataset):
    """
    A PyTorch Dataset class for sequences of tokens from song lyrics.
    
    This class prepares sequences of a specified length from the tokenized lyrics,
    suitable for training models on sequence prediction tasks, such as language modeling.
    
    Parameters:
        series (pd.Series): A pandas series containing lists of tokenized lyrics.
        seq_length (int): The length of the sequences to be prepared for the model.
    """
    def __init__(self, series, seq_length):
        self.series = series
        self.seq_length = seq_length
        self.tokens = self.prepare_tokens()

    def prepare_tokens(self):
        """
        Flattens all token lists from the series into a single list.
        
        Returns:
            list: A list of all tokens from the series combined.
        """
        # Efficient flattening using itertools.chain
        from itertools import chain
        return list(chain.from_iterable(self.series))

    def __getitem__(self, idx):
        """
        Retrieves a sequence of tokens and its corresponding target sequence from the dataset.
        
        The target sequence is the original sequence shifted by one token to the right.
        
        Parameters:
            idx (int): The start index of the sequence.
            
        Returns:
            tuple: A tuple containing the input sequence and target sequence tensors.
        """
        # Ensure the sequence does not exceed the tokens list's bounds
        end_idx = min(idx + self.seq_length, len(self.tokens) - 1)
        
        # Sequence from 'idx' to 'end_idx'
        seq = self.tokens[idx:end_idx]
        # Target sequence is offset by one token
        target_seq = self.tokens[idx + 1:end_idx + 1]

        # Padding for sequences shorter than 'seq_length' at the end of the tokens list
        seq += [0] * (self.seq_length - len(seq))  # Assuming 0 is the index for '<PAD>'
        target_seq += [0] * (self.seq_length - len(target_seq))  # Assuming 0 is the index for '<PAD>'

        # Conversion to tensors
        seq_tensor = torch.tensor(seq, dtype=torch.long)
        target_tensor = torch.tensor(target_seq, dtype=torch.long)

        return seq_tensor, target_tensor

    def __len__(self):
        """
        Returns the adjusted length of the dataset accounting for the sequence length.
        
        This adjustment ensures that each sequence has a corresponding target sequence.
        
        Returns:
            int: The adjusted length of the dataset.
        """
        # Adjust length to ensure there's a target for each input sequence
        return max(0, len(self.tokens) - self.seq_length)


In [47]:
GPT
The snippet provided outlines the initialization and basic setup for a BiLSTM model in PyTorch, including dataset preparation and dataloader configuration. It also includes a function for determining vocabulary size from a series of tokenized songs. Let's go over the components and enhance the code for clarity, best practices, and efficiency.

Enhanced BiLSTM Class Definition
python
Copy code
import torch.nn as nn

class BiLSTM(nn.Module):
    """
    A Bidirectional LSTM model for sequence prediction tasks.
    
    Parameters:
        vocab_size (int): The size of the vocabulary.
        embedding_dim (int): The dimensionality of the embedding layer.
        hidden_dim (int): The dimensionality of the LSTM's hidden states.
    """
    def __init__(self, vocab_size, embedding_dim, hidden_dim):
        super(BiLSTM, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.dropout = nn.Dropout(p=0.5)  # Adjust the dropout rate as needed
        # Ensure hidden_dim is divisible by 2 for the bidirectional LSTM
        assert hidden_dim % 2 == 0, "Hidden dimension must be even for BiLSTM."
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2, num_layers=2, dropout=0.5, bidirectional=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, sequence):
        """
        Defines the forward pass of the model.
        
        Parameters:
            sequence (Tensor): The input sequence to the model.
            
        Returns:
            Tensor: The output predictions for the next token in the sequence.
        """
        embedded = self.embedding(sequence)
        embedded = self.dropout(embedded)
        lstm_out, _ = self.lstm(embedded)
        output = self.fc(lstm_out)
        return output



def get_vocab_size_from_series(tokenized_songs_series):
    """
    Determines the vocabulary size from a series of tokenized songs.
    
    Parameters:
        tokenized_songs_series (pd.Series): A series of tokenized songs.
        
    Returns:
        int: The size of the vocabulary.
    """
    vocabulary = set().union(*tokenized_songs_series)
    return len(vocabulary)

# Assuming 'build_vocab' and 'tokens_to_indices' functions are defined as previously discussed

# Build vocabulary from tokenized series and convert songs to indices
token_to_idx = build_vocab(spacy_tokens_df)
indexed_tokens_series = spacy_tokens_df.apply(lambda song: tokens_to_indices(song, token_to_idx))

# Define the sequence length and initialize the dataset
seq_length = 70
dataset = TokenSongsDataset(indexed_tokens_series, seq_length)

# Initialize DataLoader with the dataset
batch_size = 12
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Initialize the BiLSTM model
embedding_dim = 100
hidden_dim = 256
model = BiLSTM(len(token_to_idx), embedding_dim, hidden_dim)

# Create a reverse mapping from indices to tokens for later use
idx_to_token = {idx: token for token, idx in token_to_idx.items()}


In [ ]:
import numpy as np
import torch

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Transfer the model to the configured device
model.to(device)

# Number of training epochs
num_of_epochs = 7

# Learning rate scheduler
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

# Lists to store per-epoch metrics
epoch_losses = []
epoch_accuracies = []
epoch_perplexities = []

# Training loop
for epoch in range(num_of_epochs):
    model.train()  # Set the model to training mode
    batch_losses = []
    batch_accuracies = []
    batch_perplexities = []

    for i, (inputs, targets) in enumerate(dataloader):
        # Transfer data to the configured device
        inputs, targets = inputs.to(device), targets.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        outputs_flattened = outputs.view(-1, outputs.size(-1))
        targets_flattened = targets.view(-1)

        # Loss calculation
        loss = criterion(outputs_flattened, targets_flattened)
        batch_losses.append(loss.item())

        # Calculate batch perplexity
        perplexity = torch.exp(loss).item()
        batch_perplexities.append(perplexity)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        # Calculate batch accuracy
        predicted_indices = outputs_flattened.max(1)[1]
        correct_predictions = predicted_indices.eq(targets_flattened).sum().item()
        accuracy = correct_predictions / targets_flattened.size(0)
        batch_accuracies.append(accuracy)

        # Log training progress
        if i % 900 == 0:
            print(f"Epoch [{epoch+1}/{num_of_epochs}], Batch [{i}], Loss: {loss.item():.4f}, Accuracy: {accuracy:.4f}, Perplexity: {perplexity:.4f}")

    # Calculate and store epoch metrics
    epoch_loss = np.mean(batch_losses)
    epoch_accuracy = np.mean(batch_accuracies)
    epoch_perplexity = np.mean(batch_perplexities)

    epoch_losses.append(epoch_loss)
    epoch_accuracies.append(epoch_accuracy)
    epoch_perplexities.append(epoch_perplexity)

    # Log epoch metrics
    print(f"End of Epoch [{epoch+1}], Average Loss: {epoch_loss:.4f}, Average Accuracy: {epoch_accuracy:.4f}, Average Perplexity: {epoch_perplexity:.4f}")
    print('=' * 100)

    # Step the learning rate scheduler
    scheduler.step()


Inputs shape: torch.Size([6, 70])
Targets shape: torch.Size([6])
Using device: cuda


In [ ]:
import torch
import numpy as np

def deterministic_sampling(logits):
    """
    Selects the token with the highest logit value.
    
    Parameters:
        logits (Tensor): The logits output from the model.
        
    Returns:
        int: The index of the token with the highest logit.
    """
    return logits.argmax().item()

def temperature_sampling(logits, temperature=1.0):
    """
    Samples a token index based on adjusted logits using a temperature parameter.
    
    Parameters:
        logits (Tensor): The logits output from the model.
        temperature (float): The temperature to adjust the logits.
        
    Returns:
        int: The index of the sampled token.
    """
    adjusted_logits = logits / temperature
    probabilities = torch.softmax(adjusted_logits, dim=0).cpu().numpy()
    return np.random.choice(len(logits), p=probabilities)

def top_k_sampling(logits, top_k=0):
    """
    Samples a token from the top K logits.
    
    Parameters:
        logits (Tensor): The logits output from the model.
        top_k (int): The number of top logits to consider for sampling.
        
    Returns:
        int: The index of the sampled token.
    """
    indices_to_remove = logits < torch.topk(logits, top_k)[0][..., -1, None]
    logits[indices_to_remove] = -float('Inf')
    probabilities = torch.softmax(logits, dim=0).cpu().numpy()
    return np.random.choice(len(logits), p=probabilities)

def top_p_sampling(logits, top_p=0.0):
    """
    Samples a token from the logits, considering only the top P cumulative probabilities.
    
    Parameters:
        logits (Tensor): The logits output from the model.
        top_p (float): The cumulative probability threshold.
        
    Returns:
        int: The index of the sampled token.
    """
    sorted_logits, sorted_indices = torch.sort(logits, descending=True)
    cumulative_probs = torch.cumsum(torch.softmax(sorted_logits, dim=0), dim=0)
    sorted_indices_to_remove = cumulative_probs > top_p
    sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
    sorted_indices_to_remove[..., 0] = 0
    indices_to_remove = sorted_indices[sorted_indices_to_remove]
    logits[indices_to_remove] = -float('Inf')
    probabilities = torch.softmax(logits, dim=0).cpu().numpy()
    return np.random.choice(len(logits), p=probabilities)

def generate_song(model, start_sequence, token_to_idx, idx_to_token, seq_length, device, max_length=100, method='none', temperature=1.0, top_k=0, top_p=0.0):
    """
    Generates a song sequence using a specified sampling method.
    
    Parameters:
        model (nn.Module): The trained model for generating sequences.
        start_sequence (list of str): The initial sequence of tokens to start generating from.
        token_to_idx (dict): Mapping from tokens to their indices.
        idx_to_token (dict): Mapping from indices back to tokens.
        seq_length (int): The sequence length that the model expects.
        device (torch.device): The device to perform the computation on.
        max_length (int, optional): The maximum length of the generated sequence. Defaults to 100.
        method (str, optional): The sampling method to use ('none', 'temperature', 'top-k', 'top-p'). Defaults to 'none'.
        temperature (float, optional): Temperature parameter for temperature sampling. Defaults to 1.0.
        top_k (int, optional): Top K parameter for top-k sampling. Defaults to 0.
        top_p (float, optional): Top P parameter for top-p sampling. Defaults to 0.0.
        
    Returns:
        list of str: The generated sequence of tokens.
    """
    model.eval()
    tokens = [token_to_idx.get(token, token_to_idx['<PAD>']) for token in start_sequence]
    sampling_methods = {
        'none': deterministic_sampling,
        'temperature': lambda logits: temperature_sampling(logits, temperature),
        'top-k': lambda logits: top_k_sampling(logits, top_k),
        'top-p': lambda logits: top_p_sampling(logits, top_p),
    }
    sampling_function = sampling_methods[method]

    with torch.no_grad():
        for _ in range(max_length):
            sequence = torch.tensor(tokens[-seq_length:], dtype=torch.long).unsqueeze(0).to(device)
            output = model(sequence)
            last_output = output[0, -1, :]

            predicted_token_idx = sampling_function(last_output)
            tokens.append(predicted_token_idx)

            if predicted_token_idx == token_to_idx.get('<ENDSONG>', token_to_idx['<PAD>']):
                break

    return [idx_to_token.get(idx, '<UNK>') for idx in tokens]

# Example usage
outputs = []
methods = ['none', 'temperature', 'top-k', 'top-p']
start_sequence = ['<startSong>']
for method in methods:
    print(f"Method: {method}")
    generated_song = generate_song(model, start_sequence, token_to_idx, idx_to_token, seq_length, device, max_length=300, method=method, temperature=0.8, top_k=10, top_p=0.8)
    generated_song_text = ' '.join(generated_song)
    outputs.append(generated_song_text)
    print(generated_song_text)
    print('-' * 50)


### we can try to mess more with the outputs